In [0]:
display(dbutils.fs.ls('/databricks-datasets/'))

In [0]:
display(dbutils.fs.ls('databricks-datasets/bikeSharing/'))

In [0]:
dbutils.fs.cp('databricks-datasets/bikeSharing/', '/Volumes/workspace/alura/c1/bikeSharing', recurse=True)

In [0]:
bs_df_hour = spark.read.csv('/Volumes/workspace/alura/c1/bikeSharing/data-001/hour.csv', header=True, sep=',', inferSchema=True)

bs_df_day = spark.read.csv('/Volumes/workspace/alura/c1/bikeSharing/data-001/day.csv', header=True, sep=',', inferSchema=True)

In [0]:
table_name_hour = "workspace.alura.bike_sharing_hour"

table_name_day = "workspace.alura.bike_sharing_day"

bs_df_hour.write.format('delta').mode('overwrite').saveAsTable(table_name_hour)
bs_df_day.write.format('delta').mode('overwrite').saveAsTable(table_name_day)

In [0]:
%sql
USE workspace.alura

In [0]:
display(spark.sql('''
          select * from bike_sharing_day limit 10
          '''))

In [0]:
from pyspark.sql import functions as F

bs_df_day = bs_df_day.withColumn('total_users', F.col('casual') + F.col('registered'))

bs_df_day\
    .select(['season', 'total_users'])\
    .groupBy('season')\
    .agg(F.sum('total_users').alias('qtd'))\
    .orderBy('season', ascending=False)\
    .show()

In [0]:
bs_df_day\
    .select(['season', 'total_users', 'yr'])\
    .groupBy(['season', 'yr'])\
    .agg(F.sum('total_users').alias('qtd'))\
    .orderBy(['season', 'yr'], ascending=False)\
    .show()

In [0]:
display(bs_df_hour)

In [0]:
bs_df_hour = bs_df_hour.withColumn('total_users', F.col('casual') + F.col('registered'))

display(bs_df_hour\
    .select(['cnt', 'hr'])\
    .groupBy('hr')\
    .agg(F.sum('cnt').alias('qtd'))\
    .orderBy('hr', ascending=False))


In [0]:
bs_df_2011_day = bs_df_day.filter(F.col('yr') == 0)
bs_df_2011_day.createOrReplaceTempView('bs_day_2011')

In [0]:
display(spark.sql('''
        SELECT weathersit, workingday, SUM(cnt) as total_rent
        FROM bs_day_2011
        GROUP BY weathersit, workingday
        ORDER BY weathersit, workingday
          '''))